# Airbnb Rio de Janeiro - Feature engineering

# 1. Introdução

Esse notebook propõe um processo de feature engineering para os [dados do Airbnb na cidade do Rio de Janeiro](https://insideairbnb.com/get-the-data/#:~:text=Rio%20de%20Janeiro%2C%20Rio%20de%20Janeiro%2C%20Brazil). 

A base de dados original foi previamente explorada e modificada durante a etapa de [análise exploratória (EDA)](https://github.com/BPRateiro/rio-airbnb-data-analysis/blob/main/notebooks/eda.ipynb). Os dados incluem informações detalhadas sobre a disponibilidade de listagens, características dos imóveis, interações de usuários, e preços. 

O objetivo principal é transformar essas informações para melhorar a capacidade preditiva de modelos de machine learning e facilitar análises mais profundas.

## 1.1 Configurações e bibliotecas

In [1]:
import pandas as pd
from summarytools import dfSummary
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.ensemble import IsolationForest
from dython import nominal
import numpy as np
import hiplot as hip
from distfit import distfit
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
from matplotlib.patches import Patch
import ast

In [ ]:
# Separar colunas numéricas e categóricas
numeric_cols = df_sumarizacao.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = df_sumarizacao.select_dtypes(exclude=['int64', 'float64']).columns

# Imputação de variáveis numéricas usando KNN Imputer
knn_imputer = KNNImputer(n_neighbors=5)
numeric_data_imputed = knn_imputer.fit_transform(df_sumarizacao[numeric_cols])
df_numeric_imputed = pd.DataFrame(numeric_data_imputed, columns=numeric_cols)

# Imputação de variáveis categóricas usando Simple Imputer
simple_imputer = SimpleImputer(strategy='constant', fill_value='Unknown')
categorical_data_imputed = simple_imputer.fit_transform(df_sumarizacao[categorical_cols])
df_categorical_imputed = pd.DataFrame(categorical_data_imputed, columns=categorical_cols)

# Combinar os dados imputados
df_final = pd.concat([df_numeric_imputed, df_categorical_imputed], axis=1)